#### Initial Setup

In [ ]:
#imported libraries
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt
import langdetect
import datetime
%matplotlib inline  
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import Imputer

import findspark; findspark.init()
import pyspark
import os
from pyspark.sql.types import *
from pyspark.sql.types import Row
import pyspark.sql.functions as func

os.environ["PYSPARK_SUBMIT_ARGS"] = (
  "--packages com.databricks:spark-csv_2.11:1.4.0 pyspark-shell"
)

#NEED TO ADD "  SPARK_DRIVER_MEMORY=5G   "  to ./conf/spark-env.sh 

try:
    sc = pyspark.SparkContext()
except Exception as e:
    print "SparkContext exists... Continuing on."
    
sqlCtx = pyspark.sql.SQLContext(sc)

#### Load files

In [ ]:
downloads = pd.read_csv('train_app_downloads.csv')
usages = pd.read_csv('train_usage.csv')
revenues = pd.read_csv('train_revenue.csv')

imputer = Imputer(missing_values=-1, strategy='median', axis=0)
category = list(set(downloads["category"].values))
imp = pd.DataFrame(columns = downloads.columns)
for cat in category:
    for dev in ["iphone","ipad"]:
        curr_df = downloads.ix[downloads["category"]==cat,:]
        curr_df = curr_df.ix[curr_df["device"]==dev,:]
        name = curr_df.columns
        df1 = curr_df.ix[:,0:5]
        df2 = pd.DataFrame(imputer.fit_transform(curr_df.ix[:,5:]))
        df2.index = df1.index
        curr_df = pd.concat([df1,df2],axis = 1)
        curr_df.columns = name 
        imp = pd.concat([imp,curr_df],axis = 0)
downloads = imp


imputer = Imputer(missing_values=-1, strategy='median', axis=0)
category = list(set(usages["category"].values))
imp = pd.DataFrame(columns = usages.columns)
for cat in category:
    #for dev in ["iphone","ipad"]:
        for metric in range(1,5):
            curr_df = usages.ix[usages["category"]==cat,:]
            #curr_df = curr_df.ix[curr_df["device"]==dev,:]
            curr_df = curr_df.ix[curr_df["metric"]==metric,:]
            name = curr_df.columns
            df1 = curr_df.ix[:,0:6]
            df2 = pd.DataFrame(imputer.fit_transform(curr_df.ix[:,6:]))
            df2.index = df1.index
            curr_df = pd.concat([df1,df2],axis = 1)
            curr_df.columns = name 
            imp = pd.concat([imp,curr_df],axis = 0)
usages = imp


#name = revenues.columns
#revenues = pd.concat([revenues.ix[:,0:5],pd.DataFrame(imputer.fit_transform(revenues.ix[:,5:]))],axis = 1)
#revenues.columns = name 

imputer = Imputer(missing_values=-1, strategy='median', axis=0)
category = list(set(revenues["category"].values))
imp = pd.DataFrame(columns = revenues.columns)
for cat in category:
    for dev in ["iphone","ipad"]:
        #for metric in range(1,5):
            curr_df = revenues.ix[revenues["category"]==cat,:]
            curr_df = curr_df.ix[curr_df["device"]==dev,:]
            #curr_df = curr_df.ix[curr_df["metric"]==metric,:]
            name = curr_df.columns
            df1 = curr_df.ix[:,0:5]
            df2 = pd.DataFrame(imputer.fit_transform(curr_df.ix[:,5:]))
            df2.index = df1.index
            curr_df = pd.concat([df1,df2],axis = 1)
            curr_df.columns = name 
            imp = pd.concat([imp,curr_df],axis = 0)
revenues = imp

downloads.to_csv("downloads.csv")
revenues.to_csv("revenues.csv")
usages.to_csv("usages.csv")

In [ ]:
downloads = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('downloads.csv').drop('')
ratings = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_app_rating.csv').drop('')
usages = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('usages.csv').drop('')
revenues = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('revenues.csv').drop('')
output = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_final_downloads.csv').drop('')
prev_downloads = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_cumulative_downloads_2015-02.csv').drop('')  
release_date = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_release_date.csv').drop('')
text_score = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='false',inferSchema='true') \
    .load('sentiment.csv').drop('')
title_score = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='false',inferSchema='true') \
    .load('t_sentiment.csv').drop('')
avg_score = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('avg_sent_score.csv').drop('')
    
    
reviews = pd.read_csv('train_app_review.csv')
reviews_schema = StructType([
    StructField("id",IntegerType(),True),
    StructField("name",StringType(),True),
    StructField("country",StringType(),True),
    StructField("rating",IntegerType(),True),
    StructField("date",StringType(),True),
    StructField("title",StringType(),True),
    StructField("version",StringType(),True),
    StructField("text",StringType(),True),
    StructField("reviewer",StringType(),True)
    
])
reviews = sqlCtx.createDataFrame(reviews,reviews_schema)

#### Generate Predictors

In [ ]:
old_dateRange = pd.date_range('2015-03-01', periods=56).format(formatter=lambda x: x.strftime('%Y-%m-%d'))
dateRange = pd.date_range('2015-03-01', periods=56).format(formatter=lambda x: x.strftime('%Y_%m_%d'))
for d in range(56):
    revenues = revenues.withColumnRenamed(old_dateRange[d],dateRange[d])
    usages = usages.withColumnRenamed(old_dateRange[d],dateRange[d])
    downloads = downloads.withColumnRenamed(old_dateRange[d],dateRange[d])

In [ ]:
#Initialization
predictors = downloads['id','name','category','device']

In [ ]:
# Generate the weekly downloads

sqlCtx.registerDataFrameAsTable(downloads, "downloads")
sqlCtx.registerDataFrameAsTable(usages, "usages")
sqlCtx.registerDataFrameAsTable(revenues, "revenues")

predictors = sqlCtx.sql("SELECT id, name, category, device, "+\
           "+".join(dateRange[0:7])+" AS week_1, "+\
           "+".join(dateRange[7:14])+" AS week_2, "+\
           "+".join(dateRange[14:21])+" AS week_3, "+\
           "+".join(dateRange[21:28])+" AS week_4, "+\
           "+".join(dateRange[28:35])+" AS week_5, "+\
           "+".join(dateRange[35:42])+" AS week_6, "+\
           "+".join(dateRange[42:49])+" AS week_7, "+\
           "+".join(dateRange[49:56])+" AS week_8 "+\
           #"+".join(dateRange)+" AS download_sum \
           "from downloads")
sqlCtx.registerDataFrameAsTable(predictors, "predictors")

#I workaround the error by this modification. 
#I don't know why I couldn't run the code before but this workaround gives the same result.
predictors = sqlCtx.sql("SELECT "+', '.join(predictors.columns)+", week_1+week_2+week_3+week_4+week_5+week_6+week_7+week_8 AS download_sum FROM predictors")


In [ ]:
m1 = sqlCtx.sql("SELECT * FROM usages WHERE metric = 1")
m2 = sqlCtx.sql("SELECT * FROM usages WHERE metric = 2")
m3 = sqlCtx.sql("SELECT * FROM usages WHERE metric = 3")
m4 = sqlCtx.sql("SELECT * FROM usages WHERE metric = 4")
sqlCtx.registerDataFrameAsTable(m1,"m1")
sqlCtx.registerDataFrameAsTable(m2,"m2")
sqlCtx.registerDataFrameAsTable(m3,"m3")
sqlCtx.registerDataFrameAsTable(m4,"m4")
sqlCtx.registerDataFrameAsTable(avg_score,"avg_score")

In [ ]:
# Make coefficients

def get_coefficients(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    return  float(np.polyfit(range(56),np.cumsum(args[1:]),args[0])[0])
    
#Generate the step max and min 
def get_maxStep(maximum,*args):
    args=list(args)
    if (np.count_nonzero(args) == 0):
        return 0
    m = 0
    for d in range(1,56):
        if (args[d]!=0 and args[d-1]!=0):
            c = (args[d]-args[d-1])
            if (maximum and m < c):
                m = c
            if ( not maximum and m > c):
                m = c
    return m

def get_std(*args):
    return float(np.std(list(args)))

def get_nbMissing(*args):
    return list(args).count(-1)
replacementValue = 0
#Generate the daily average
def get_dailyAvg(*inp):
    if (np.count_nonzero(inp - replacementValue*np.ones(len(inp))) == 0):
        return 0
    return  (1.0*sum(inp)/np.count_nonzero(inp - replacementValue*np.ones(len(inp))))

def get_usage_coefficients(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    if -1 in args: return 0
    return  float(np.polyfit(range(8),args[1:],args[0])[0])

def get_revenue_coefficients(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    if -1 in args: return 0
    return  float(np.polyfit(range(56),args[1:],args[0])[0])

def get_usage_max(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    return  float(args[1:].max())

def get_usage_mean(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    if -1 in args: return 0
    return  float(args[1:].mean())

def get_revenue_max(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    if -1 in args: return 0
    return  float(args[1:].max())

def get_revenue_mean(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    if -1 in args: return 0
    return  float(args[1:].max())

sqlCtx.registerFunction("get_nbMissing", get_nbMissing,returnType=IntegerType())
sqlCtx.registerFunction("get_std", get_std,returnType=FloatType())
sqlCtx.registerFunction("get_maxStep", get_maxStep,returnType=IntegerType())
sqlCtx.registerFunction("get_coefficients", get_coefficients,returnType=FloatType())
sqlCtx.registerFunction("daily_avg", get_dailyAvg,returnType=FloatType())
sqlCtx.registerFunction("get_usage_coefficients", get_usage_coefficients,returnType=FloatType())
sqlCtx.registerFunction("get_revenue_coefficients", get_revenue_coefficients,returnType=FloatType())
sqlCtx.registerFunction("get_usage_max", get_usage_max,returnType=FloatType())
sqlCtx.registerFunction("get_usage_mean", get_usage_mean,returnType=FloatType())
sqlCtx.registerFunction("get_revenue_max", get_revenue_max,returnType=FloatType())
sqlCtx.registerFunction("get_revenue_mean", get_revenue_mean,returnType=FloatType())

temp_downloads = sqlCtx.sql("SELECT id,name,category, device \
, get_coefficients(0,"+",".join(dateRange)+") AS coef_0 \
,get_coefficients(1,"+",".join(dateRange)+") AS coef_1 \
,get_coefficients(2,"+",".join(dateRange)+") AS coef_2 \
,get_coefficients(3,"+",".join(dateRange)+") AS coef_3 \
,get_maxStep(True,"+",".join(dateRange)+") AS max_step \
,get_maxStep(False,"+",".join(dateRange)+") AS min_step \
,get_std("+",".join(dateRange)+") AS downloads_std \
,get_nbMissing("+",".join(dateRange)+") AS nb_missing \
,daily_avg(" + ",".join(dateRange[0:56]) + ") AS daily_avg \
 FROM downloads")

temp_m1 = sqlCtx.sql("SELECT id, name, category, device, \
get_usage_coefficients(0,"+",".join(m1.columns[5:13])+") AS m1_coef_0, \
get_usage_coefficients(1,"+",".join(m1.columns[5:13])+") AS m1_coef_1, \
get_usage_coefficients(2,"+",".join(m1.columns[5:13])+") AS m1_coef_2, \
get_usage_max(0,"+",".join(m1.columns[5:13])+") AS m1_max, \
get_usage_mean(0,"+",".join(m1.columns[5:13])+") AS m1_mean FROM m1")

temp_m2 = sqlCtx.sql("SELECT id, name, category, device, \
get_usage_coefficients(0,"+",".join(m2.columns[5:13])+") AS m2_coef_0, \
get_usage_coefficients(1,"+",".join(m2.columns[5:13])+") AS m2_coef_1, \
get_usage_coefficients(2,"+",".join(m2.columns[5:13])+") AS m2_coef_2, \
get_usage_max(0,"+",".join(m2.columns[5:13])+") AS m2_max, \
get_usage_mean(0,"+",".join(m2.columns[5:13])+") AS m2_mean FROM m2")

temp_m3 = sqlCtx.sql("SELECT id, name, category, device, \
get_usage_coefficients(0,"+",".join(m3.columns[5:13])+") AS m3_coef_0, \
get_usage_coefficients(1,"+",".join(m3.columns[5:13])+") AS m3_coef_1, \
get_usage_coefficients(2,"+",".join(m3.columns[5:13])+") AS m3_coef_2, \
get_usage_max(0,"+",".join(m3.columns[5:13])+") AS m3_max, \
get_usage_mean(0,"+",".join(m3.columns[5:13])+") AS m3_mean FROM m3")

temp_m4 = sqlCtx.sql("SELECT id, name, category, device, \
get_usage_coefficients(0,"+",".join(m4.columns[5:13])+") AS m4_coef_0, \
get_usage_coefficients(1,"+",".join(m4.columns[5:13])+") AS m4_coef_1, \
get_usage_coefficients(2,"+",".join(m4.columns[5:13])+") AS m4_coef_2, \
get_usage_max(0,"+",".join(m4.columns[5:13])+") AS m4_max, \
get_usage_mean(0,"+",".join(m4.columns[5:13])+") AS m4_mean FROM m4")

temp_revenues = sqlCtx.sql("SELECT id, name, category, device, \
get_revenue_coefficients(0,"+",".join(revenues.columns[4:])+") AS rev_coef_0, \
get_revenue_coefficients(1,"+",".join(revenues.columns[4:])+") AS rev_coef_1, \
get_revenue_coefficients(2,"+",".join(revenues.columns[4:])+") AS rev_coef_2, \
get_revenue_max(0,"+",".join(revenues.columns[4:])+") AS rev_max, \
get_revenue_mean(0,"+",".join(revenues.columns[4:])+") AS rev_mean FROM revenues")

predictors = predictors.join(temp_downloads,["id","name","category","device"])
predictors = predictors.join(temp_revenues,["id", "name", "category","device"])
predictors = predictors.join(temp_m1,["id", "name", "category","device"])
predictors = predictors.join(temp_m2,["id", "name", "category","device"])
predictors = predictors.join(temp_m3,["id", "name", "category","device"])
predictors = predictors.join(temp_m4,["id", "name", "category","device"])
predictors = predictors.join(avg_score,["id", "name", "category","device"])
#predictors = predictors.join(temp_usages,["id", "name", "category","device"])
#predictors = predictors.join(,)



In [ ]:
# previous downloads addition
predictors = predictors.join(prev_downloads,["id","device"])

In [ ]:
# Days since release generation
def get_days(date):
    return (datetime.datetime.strptime('03/01/2015', '%m/%d/%Y').date() \
            - datetime.datetime.strptime(date, '%m/%d/%Y').date()).days

sqlCtx.registerDataFrameAsTable(release_date, "release_date")
sqlCtx.registerFunction("get_days", get_days,returnType=IntegerType())
temp_date = sqlCtx.sql("SELECT id,name \
, get_days(release_date) AS days_since_release \
 FROM release_date")

predictors = predictors.join(temp_date,["id","name"],"left")

In [ ]:
#ratings generation
sqlCtx.registerDataFrameAsTable(ratings, "ratings")
temp_ratings = sqlCtx.sql("SELECT id,name,category \
, start1/(start1+star2+star3+star4+star5) AS star1 \
, star2/(start1+star2+star3+star4+star5) AS star2 \
, star3/(start1+star2+star3+star4+star5) AS star3 \
, star4/(start1+star2+star3+star4+star5) AS star4 \
, star5/(start1+star2+star3+star4+star5) AS star5 \
, (start1+star2+star3+star4+star5) AS num_ratings \
 FROM ratings")

predictors = predictors.join(temp_ratings,["id","name","category"],"left")

In [ ]:
# Categories
list_categories = [ x.category.replace(" ","_") for x in sqlCtx.sql("SELECT category \
 FROM downloads\
 group by category \
 ").collect()]
for cat in list_categories:
    sqlCtx.registerDataFrameAsTable(predictors, "predictors")
    predictors=sqlCtx.sql('''SELECT *, CASE WHEN (category = "'''+cat+'''") THEN 1 ELSE 0 END AS '''+cat+''' FROM predictors''')


In [ ]:
# Device
sqlCtx.registerDataFrameAsTable(predictors, "predictors")
predictors=sqlCtx.sql('''SELECT *, CASE WHEN (device = "iphone") THEN 1 ELSE 0 END AS iphone FROM predictors''')
sqlCtx.registerDataFrameAsTable(predictors, "predictors")
predictors=sqlCtx.sql('''SELECT *, CASE WHEN (device = "ipad") THEN 1 ELSE 0 END AS ipad FROM predictors''')

In [ ]:
predictors

In [ ]:
def get_language(x):
    try:
        detected = langdetect.detect_langs(x.decode('utf8','ignore'))[0]
        if detected.prob < 0.7:
            return "other"
        else :
            return detected.lang
    except:
        return "other"
sqlCtx.registerFunction("get_language", get_language,returnType=StringType())

In [ ]:
#Language of the title
lang = ['ja','zh-cn','ko','en']
for l in lang:
    sqlCtx.registerDataFrameAsTable(predictors, "predictors")
    predictors=sqlCtx.sql('''SELECT *, CASE WHEN (get_language(name) = "'''+l+'''") THEN 1 \
    ELSE 0 END AS '''+l.replace("-","_")+''' FROM predictors''')

In [ ]:
#Reviews 
#escape is used in case some asshole used - or [space] anywhere
def escape(text):
    return text.replace(" ","_").replace("-","_")
# number of reviews
def get_recentReviews(date):
    return int((datetime.datetime.strptime('03/01/2015', '%m/%d/%Y').date() \
            - datetime.datetime.strptime(date, '%Y-%m-%d').date()).days >=0)

#First step
list_countries =['United_States', 'France', 'Japan', 'Spain', 'United_Kingdom','Saudi_Arabia', 'Germany'\
     , 'Hong_Kong', 'Switzerland', 'Turkey','Netherlands', 'Australia', 'Norway', 'Sweden', 'China', 'Canada'\
     ,'Tanzania', 'Denmark', 'South_Korea', 'Italy', 'Finland', 'Taiwan','Russia', 'Philippines', 'Slovenia'\
     , 'Ireland', 'Belgium', 'Mexico','Austria', 'India', 'Brazil', 'Benin', 'New_Zealand','United_Arab_Emirates'\
     , 'Ukraine', 'Poland', 'Israel', 'Portugal','Tunisia', 'Mali', 'Slovakia', 'Zimbabwe', 'Thailand', 'Panama'\
     ,'Indonesia', 'Singapore', 'Greece', 'Senegal', 'Nicaragua','Hungary', 'Czech_Republic', 'Macedonia', 'Chile'\
     , 'Uruguay','Malaysia', 'Algeria', 'Nepal', 'Mauritania', 'Croatia']

cmd = '''review_rdd = reviews\
.map(lambda x : (x.id , Row(id = x.id , avg_review = x.rating \
, recent_review = get_recentReviews(x.date), nb_review = 1\
,version = set([x.version])'''
for c in list_countries:
    cmd+=","+c+''' = int( escape(x.country) == "'''+c+'''")'''
cmd+=")))"
exec cmd

#Group step
cmd = '''review_rdd = review_rdd.reduceByKey(lambda x1 ,x2 : Row(\
 avg_review = x1.avg_review + x2.avg_review\
   ,recent_review = x1.recent_review + x2.recent_review, nb_review = x1.nb_review + x2.nb_review'''
for c in list_countries:
    cmd+=" , "+c+" = x1."+c+" + x2."+c
cmd+=", version = x1.version.union(x2.version)))"
exec cmd

# Clean the grouped rdd
cmd = '''review_rdd = review_rdd.map(lambda (id , x) : [ id \
,  1.0*x.avg_review /  x.nb_review\
   , x.recent_review,  x.nb_review'''
for c in list_countries:
    cmd+=" , 1.0* x."+c+"/ x.nb_review"
cmd+=",  len(x.version) - 1])"
exec cmd

#Put back into dataframe
grp_reviews = sqlCtx.createDataFrame(review_rdd, ["id","avg_review"\
             ,"recent_reviews","nb_review"] + list_countries + ["versions"])

#Join with predictors 
predictors = predictors.join(grp_reviews,["id"],"left")

In [ ]:
# Generate DL Projection
sqlCtx.registerDataFrameAsTable(predictors, "predictors")
dl_projection = sqlCtx.sql("SELECT id, device, (download_sum+7*`cumulative_downloads_2015-02`) AS dl_projection \
                                FROM predictors")
predictors = predictors.join(dl_projection,["id","device"],"left")

In [ ]:
predictors_Pandas = predictors.toPandas()

In [ ]:
predictors_Pandas

In [ ]:
predictors.head(1)

In [ ]:
release_date.head(1)